In [1]:
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(0,"..")
from src_dual.EIT_kernel import performance

In [2]:
base_path="../eit_basic_dual_unreduced_experiments/"

In [3]:
!ls $base_path

exp_1      exp_1_npca exp_2      exp_2b     exp_2e
exp_1_nmf  exp_1basic exp_2a     exp_2d     exp_3


In [4]:
path_reduced=base_path+"/exp_1_npca/"

In [5]:
!ls $path_reduced

basic                   combined_k_pho_nuh.csv  dual_results_exp_1.csv
basic_results_exp_1.csv dual


In [6]:
basic_results=pd.read_csv(path_reduced+"basic_results_exp_1.csv")

In [7]:
def read_reduced_result(output,bucket,file_indent):
    if bucket>=1:
        file=output+f"/EIT_bucket_{int(bucket)}_result_index_{file_indent}.csv"
    else:
        file=output+f"/EIT_kernel_result_index_{file_indent}.csv"
    result=pd.read_csv(file)
    return (result)

In [58]:
def transpose_to_unreduced(df):
    output=df["output"]
    bucket=df["bucket"]
    file_indent=df["file"]
    z_value=df["z_value"]
    T=df["T"]
    if np.isnan(z_value):
        return (None,None,None,None,None)
    else:
        #Step 1: Read required files
        reduced_result=read_reduced_result(output,bucket,file_indent)
        price_reduced=pd.read_csv(f"../input/index-weekly-data/index_{file_indent}.csv")
        price_not_reduced=pd.read_csv("../input/index-weekly-data/index_1.csv")
        #Step_2: Get Weights
        q_T = price_reduced[reduced_result.security].iloc[T]
        w = reduced_result["X_1"].values * q_T.values
        w = (w / np.sum(w))
        #Step 3: Get new X_1
        q_T_new = price_not_reduced[reduced_result.security].iloc[200]
        X_1_new = w *q_T_new
        new_result = reduced_result.copy()
        new_result.index=new_result.security
        new_result["X_1"] = X_1_new
        #Generate performance
        in_excess_return, in_tr_err, out_excess_return, out_tr_err, portfolio_size = \
            performance(price_not_reduced, new_result, C=100000, T=200)
        return(in_excess_return, in_tr_err, out_excess_return, out_tr_err, portfolio_size)

In [63]:
def transpose_df(super_df):
    super_df["chk"]=super_df.apply(lambda x: transpose_to_unreduced(x),axis=1)
    transpose_cols=["in_excess_return_transpose","in_tr_err_transpose","out_excess_return_transpose",\
               "out_tr_err_transpose","portfolio_size_transpose"]
    super_df[transpose_cols]=pd.DataFrame(super_df["chk"].to_list())
    return (super_df)

In [59]:
basic_results["chk"]=basic_results.apply(lambda x: transpose_to_unreduced(x),axis=1)

In [62]:
transpose_cols=["in_excess_return_transpose","in_tr_err_transpose","out_excess_return_transpose",\
               "out_tr_err_transpose","portfolio_size_transpose"]
basic_results[transpose_cols]=pd.DataFrame(basic_results["chk"].to_list())

Index(['Unnamed: 0', 'output', 'T', 'C', 'file', 'm', 'lbuck', 'pho', 'lamda',
       'nuh', 'xii', 'k', 'p', 'f', 'index', 'bucket', 'kernel_size',
       'problem_status', 'z_value', 'in_excess_return', 'in_tr_err',
       'out_excess_return', 'out_tr_err', 'portfolio_size', 'z_linear', 'chk'],
      dtype='object')

In [46]:
new_result.head()

,security,X_0,y,w,b,s,X_1
security,,,,,,,
security_10,security_10,2.962947,1.0,1.0,0.000000,1.515295,1637.689184
security_28,security_28,0.204695,1.0,1.0,0.834952,0.000000,209.220642
security_29,security_29,4.218816,1.0,1.0,0.000000,0.048468,1382.809858
security_7,security_7,0.474302,1.0,0.0,0.000000,0.000000,273.862752
security_19,security_19,0.434461,1.0,1.0,0.090375,0.000000,917.134175


In [149]:
%pdb
performance(price_not_reduced, new_result, C=100000, T=200)

Automatic pdb calling has been turned ON


../src_dual/EIT_kernel.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  return_scaling = C/((result["X_1"] * out_q_T).sum()) #B'cos not entire C might be invested


(nan, 79315.31386676161, nan, 124758.42458861685, 0)

In [150]:
price=price_not_reduced

In [151]:
in_price = price[0:T + 1]
out_price = price[T + 1:]
in_returns = (in_price - in_price.shift(1)) / in_price.shift(1)
in_returns.drop([0], axis=0, inplace=True)
out_returns = (out_price - out_price.shift(1)) / out_price.shift(1)
out_returns.drop([T + 1], axis=0, inplace=True)

In [159]:
new_result["X_1"]=X_1_new

In [160]:
new_result

,security,X_0,X_1,y,w,b,s
security,,,,,,,
security_10,security_10,2231.148826,2.831510,1.0,1.0,0.000000,15510.875744
security_7,security_7,243.617332,3.968673,1.0,1.0,2089.821464,0.000000
security_11,security_11,268.269085,2.475396,1.0,1.0,3534.019236,0.000000
security_26,security_26,91.529664,11.940461,1.0,1.0,0.000000,1537.168677
security_21,security_21,102.897731,6.469270,1.0,1.0,0.000000,864.819046
security_28,security_28,97.257674,4.192881,1.0,1.0,489.297239,0.000000
security_19,security_19,682.866763,0.664947,1.0,1.0,0.000000,1029.257542
security_14,security_14,187.788360,1.274668,1.0,1.0,2152.288573,0.000000
security_15,security_15,52.712522,11.170767,1.0,1.0,0.000000,5568.547855


In [ ]:
result.index = result.security
# Get In sample and out of sample Excess Return
in_q_T = in_price.iloc[-1]
out_q_T = in_q_T
y_in = result["X_1"] * in_returns * in_q_T #Stock Return at times t
y_out = result["X_1"] * out_returns * out_q_T
return_scaling = C/((result["X_1"] * out_q_T).sum()) #B'cos not entire C might be invested
in_excess_return = ((y_in.sum(axis=1) * return_scaling) - (in_returns["index"] * C)).mean()
out_excess_return = ((y_out.sum(axis=1) * return_scaling) - (out_returns["index"] * C)).mean()
z_in = result["X_1"] * in_price # Value of stocks in portfolio at time =t
z_out = result["X_1"] * out_price
theta=C / in_price["index"].iloc[-1]
in_tr_err = abs((theta * in_price["index"]) - (z_in.sum(axis=1))).mean()
out_tr_err = abs((theta * out_price["index"]) - (z_out.sum(axis=1))).mean()
portfolio_size=(result["X_1"]>0).sum()